In [1]:
from tests.models import Pic, Product, Description

In [7]:
# Если нужно воспользоваться моделями, прогоняем миграции (если их нет так же создаём)
import sys
from django.core.management import execute_from_command_line
execute_from_command_line(['manage.py', 'migrate', 'tests'])

CREATE TABLE "django_migrations" ("id" integer NOT NULL PRIMARY KEY AUTOINCREMENT, "app" varchar(255) NOT NULL, "name" varchar(255) NOT NULL, "applied" datetime NOT NULL); (params None)


Operations to perform:
  Apply all migrations: tests
Running migrations:
  Applying tests.0001_initial...

CREATE TABLE "tests_pic" ("id" integer NOT NULL PRIMARY KEY AUTOINCREMENT, "img" text NOT NULL); (params None)
CREATE TABLE "tests_size" ("id" integer NOT NULL PRIMARY KEY AUTOINCREMENT, "name" text NOT NULL, "picture_id" bigint NULL REFERENCES "tests_pic" ("id") DEFERRABLE INITIALLY DEFERRED); (params None)
CREATE TABLE "tests_product" ("id" integer NOT NULL PRIMARY KEY AUTOINCREMENT, "name" text NOT NULL, "picture_id" bigint NOT NULL REFERENCES "tests_pic" ("id") DEFERRABLE INITIALLY DEFERRED); (params None)
CREATE TABLE "tests_product_sizes" ("id" integer NOT NULL PRIMARY KEY AUTOINCREMENT, "product_id" bigint NOT NULL REFERENCES "tests_product" ("id") DEFERRABLE INITIALLY DEFERRED, "size_id" bigint NOT NULL REFERENCES "tests_size" ("id") DEFERRABLE INITIALLY DEFERRED); (params None)
CREATE TABLE "tests_description" ("id" integer NOT NULL PRIMARY KEY AUTOINCREMENT, "text" text NOT NULL, "product_id" bigint NOT NULL UNIQUE REFERENCES "tests_product" ("id") DEFERRABLE INITIALLY DEFERR

 OK


### **django.db.models.fields.related_descriptors.ManyToManyDescriptor**

In [65]:
Product.sizes  # Возвращает объект ManyToManyDescriptor

In [66]:
Product.sizes.__dict__  # имеет следующие атрибуты, нас интересует rel

{'rel': <ManyToManyRel: tests.product>,
 'field': <django.db.models.fields.related.ManyToManyField: sizes>,
 'reverse': False}

In [67]:
Product.sizes.rel.model
#  из атрибута rel можем получить модель, с которой m2m связь (может понадобится при генерации отчёта)

tests.models.Size

In [68]:
# важно учитывать, что обратная связь в fields подаётся как
# Size.objects.values('product__name')
# Однако доступ к дескриптору получаем через аттрибут field +'_set'
Size.product_set

In [69]:
# а доступ к модели в таком случае
Size.product_set.rel.related_model

tests.models.Product

In [22]:
### **django.db.models.fields.related_descriptors.ReverseManyToOneDescriptor**

In [26]:
Pic.size_set

In [27]:
Pic.size_set.__dict__

{'rel': <ManyToOneRel: tests.size>,
 'field': <django.db.models.fields.related.ForeignKey: picture>}

In [2]:
Pic.size_set.rel.related_model

tests.models.Size

In [30]:
Pic.size_set.field.model

tests.models.Size

In [24]:
Size.objects.select_related('picture').query.__str__()

'SELECT "tests_size"."id", "tests_size"."name", "tests_size"."picture_id", "tests_pic"."id", "tests_pic"."img" FROM "tests_size" LEFT OUTER JOIN "tests_pic" ON ("tests_size"."picture_id" = "tests_pic"."id")'

#### Сверху то что префетчим

#### Снизу то что джоиним

### **django.db.models.fields.related_descriptors.ForwardManyToOneDescriptor**

In [73]:
Product.picture

In [74]:
Product.picture.__dict__  # При наличии ForwardManyToOneDescriptor нам понадобится джоин в sql
# С помощью select_related, потому что на один объект главной модели приходится один объект связанный по
# внешнему ключу, в аргументы select_related должна пойти строка "picture" (название поля у модели)

{'field': <django.db.models.fields.related.ForeignKey: picture>}

In [75]:
# Доступ к модели
Product.picture.field.related_model

tests.models.Pic

### **django.db.models.fields.related_descriptors.ReverseOneToOneDescriptor**

In [76]:
print("Дескриптор: ", Product.description)
print('Его атрибуты: ', Product.description.__dict__)
print("Модель: ", Product.description.related.related_model)

Дескриптор:  <django.db.models.fields.related_descriptors.ReverseOneToOneDescriptor object at 0x7f8d3f3221a0>
Его атрибуты:  {'related': <OneToOneRel: tests.description>}
Модель:  <class 'tests.models.Description'>


### **django.db.models.fields.related_descriptors.ForwardOneToOneDescriptor**

In [77]:
print("Дескриптор: ", Description.product)
print('Его атрибуты: ', Description.product.__dict__)
print("Модель: ", Description.product.field.related_model)

Дескриптор:  <django.db.models.fields.related_descriptors.ForwardOneToOneDescriptor object at 0x7f8d3f321db0>
Его атрибуты:  {'field': <django.db.models.fields.related.OneToOneField: product>}
Модель:  <class 'tests.models.Product'>


In [2]:
def get_queryset_builder(select_related, prefetch_related, annotate_fields):
        
    def func(self):
        return self.queryset
    
    methods = {}
    if select_related:
        def select_related_decorator(func):
            def wrapper(self):
                return func(self).select_related(*select_related)
            return wrapper
        func = select_related_decorator(func)
    if prefetch_related:
        def prefetch_related_decorator(func):
            def wrapper(self):
                return func(self).prefetch_related(*prefetch_related)
            return wrapper
        func = prefetch_related_decorator(func)
    if annotate_fields:
        def annotate_decorator(func):
            def wrapper(self):
                return func(self).annotate(**annotate_fields)
            return wrapper
            
    return func

In [3]:
get_queryset_builder(['a'], None)

<function __main__.get_queryset_builder.<locals>.select_related_decorator.<locals>.wrapper(self)>

In [14]:
class A: pass
A.f = get_queryset_builder(['description'], ['sizes'])
a = A()
a.queryset = Product.objects.all()

In [15]:
a.f().query.__dict__

{'model': tests.models.Product,
 'alias_refcount': {},
 'alias_map': {},
 'alias_cols': True,
 'external_aliases': {},
 'table_map': {},
 'used_aliases': set(),
 'where': <WhereNode: (AND: )>,
 'annotations': {},
 'extra': {},
 '_filtered_relations': {},
 '_annotation_select_cache': None,
 'filter_is_sticky': False,
 'select_related': {'description': {}}}

In [21]:
from django.conf import settings
for i in a.f():
    print(i)
    print(i.sizes.all())

{'_wrapped': <Settings "tests.settings">, 'LOGGING_CONFIG': 'logging.config.dictConfig', 'LOGGING': {'version': 1, 'handlers': {'console': {'class': 'logging.StreamHandler'}}, 'loggers': {'django': {'level': 'INFO'}, 'django.db.backends': {'level': 'DEBUG'}}, 'root': {'handlers': ['console']}}, 'DEFAULT_EXCEPTION_REPORTER': 'django.views.debug.ExceptionReporter', 'FORCE_SCRIPT_NAME': None, 'INSTALLED_APPS': ('django_extensions', 'tests'), 'DEFAULT_TABLESPACE': '', 'DATABASES': {'default': {'ENGINE': 'django.db.backends.sqlite3', 'NAME': ':memory:', 'ATOMIC_REQUESTS': False, 'AUTOCOMMIT': True, 'CONN_MAX_AGE': 0, 'CONN_HEALTH_CHECKS': False, 'OPTIONS': {}, 'TIME_ZONE': None, 'USER': '', 'PASSWORD': '', 'HOST': '', 'PORT': '', 'TEST': {'CHARSET': None, 'COLLATION': None, 'MIGRATE': True, 'MIRROR': None, 'NAME': None}}}, 'DEFAULT_AUTO_FIELD': 'django.db.models.BigAutoField', 'ABSOLUTE_URL_OVERRIDES': {}, 'USE_I18N': True, 'DEBUG': False, 'LANGUAGE_CODE': 'en-us', 'LOCALE_PATHS': [], 'DEFA

In [29]:
Description.objects.select_related('product__picture').query.__str__()

'SELECT "tests_description"."id", "tests_description"."text", "tests_description"."product_id", "tests_product"."id", "tests_product"."name", "tests_product"."picture_id", "tests_pic"."id", "tests_pic"."img" FROM "tests_description" INNER JOIN "tests_product" ON ("tests_description"."product_id" = "tests_product"."id") INNER JOIN "tests_pic" ON ("tests_product"."picture_id" = "tests_pic"."id")'